# Adaptive

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
import numpy as np
import learner1D
from time import sleep
from random import randint
from functools import partial
import importlib
importlib.reload(learner1D)

def func(x, wait=False):
    """Function with a sharp peak on a smooth background"""
    x = np.asarray(x)
    a = 0.001
    if wait:
        sleep(np.random.rand(1)/10)
    return x + a**2/(a**2 + x**2) #+ np.random.rand(1)

def plot(learner, nan_is_zero=False, show_interp=False):
    if show_interp:
        learner.interpolate()
        d = learner.interp_data
    else:
        d = learner.data

    xy = [(k, d[k]) for k in sorted(d)]
    x, y  = np.array(xy, dtype=float).T

    return hv.Scatter((x, y))

$$x + a^2/(a^2 + x^2)$$
$$a = 0.001$$

In [ ]:
xs = np.linspace(-1, 1, 10)
ys = func(xs)
learner = learner1D.Learner1D(xs, ys)
plot(learner)[-1.1:1.1, -1.1:1.1]

In [ ]:
xs = learner.choose_points(n=10)
ys = func(xs)
learner.add_data(xs, ys)
plot(learner)[-1.1:1.1, -1.1:1.1]

In [ ]:
xs = learner.choose_points(n=30)
# Do not calculate ys here.
plot(learner, show_interp=True)[-1.1:1.1, -1.1:1.1]

# Parallel

In [ ]:
from dask import delayed
from distributed import Client
client = Client()
num_cores = sum(client.ncores().values())
num_cores

In [ ]:
func2 = partial(func, wait=True)
learner = learner1D.Learner1D(client=client)
learner.initialize(func2, -1, 1)

while True:
    if len(client.futures) < num_cores:
        xs = learner.choose_points(n=1)
        learner.map(func, xs)
    if len(learner.data) > 100: # bad criterion
        break
plot(learner)

In [ ]:
func2 = partial(func, wait=True)
learner = learner1D.Learner1D(client=client)
learner.initialize(func2, -1, 1)

while True:
    if len(client.futures) < num_cores:
        xs = learner.choose_points(n=1)
        learner.map(func, xs)
    if len(learner.get_done()) > 150: # bad criterion
        break
plot(learner)

In [ ]:
func_wait = partial(func, wait=True)
learner = learner1D.Learner1D(client=client)
learner.initialize(func_wait, -1, 1)

while True:
    if len(client.futures) < num_cores:
        xs = learner.choose_points(n=1)
        learner.map(func_wait, xs)
    if learner.get_largest_interval() < 0.01 * learner.x_range:
        break

print(len(learner.data), len(client.futures))
plot(learner)

## Desired interface
```python
async = learner.async_map(funcs, xs, tol=0.01)
if async.done():
    print('done')
```

In [ ]:
xs = np.linspace(-1, 1, 5000)
ys = func(xs)
learner = learner1D.Learner1D(xs, ys)
plot(learner)[-1.1:1.1, -1.1:1.1]